# Data Analysis on Brazilian E-Commerce Public Dataset by Olist

Dataset link: https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce?select=olist_customers_dataset.csv

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("FinalProject")\
        .config('spark.driver.extraClassPath','/usr/lib/jvm/java-11-openjdk-amd64/lib/postgresql-42.6.0.jar')\
        .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/12 15:36:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Loading the dataset

In [3]:
# Defining path to the dataset
customer_data_path = "./Data/olist_customers_dataset.csv"  
order_item_path = "./Data/olist_order_items_dataset.csv"
order_payment_path = "./Data/olist_order_payments_dataset.csv"
product_category_translation_path= "./Data/product_category_name_translation.csv"
product_path = './Data/olist_products_dataset.csv'
seller_path = './Data/olist_sellers_dataset.csv'
orders_path = './Data/olist_orders_dataset.csv'
review_path = "Data/reviews_translated.csv"  


# Load the Chipotle dataset into a Spark DataFrame
customer_df = spark.read.csv(customer_data_path, header=True, inferSchema=True)
order_item_df = spark.read.csv(order_item_path, header=True, inferSchema=True)
order_payment_df = spark.read.csv(order_payment_path, header=True, inferSchema=True)
product_category_translation_df = spark.read.csv(product_category_translation_path, header=True, inferSchema=True)
seller_df_uncleaned = spark.read.csv(seller_path, header=True, inferSchema=True)
product_df_uncleaned = spark.read.csv(product_path, header=True, inferSchema=True)
orders_df = spark.read.csv(orders_path, header=True, inferSchema= True)
reviews_df = spark.read.csv(review_path, header=True, inferSchema= True)

# Data Cleaning and pre-processing

In [4]:
from pyspark.sql import functions as f

### Removing whitespace  

In [5]:
# Remove leading and trailing whitespace from all columns
seller_df_uncleaned.select([f.trim(f.col(c)).alias(c) for c in seller_df_uncleaned.columns])

# Remove whitespace characters between words in all columns
seller_df = seller_df_uncleaned.select([f.regexp_replace(f.col(c), r'\s+', ' ').alias(c) for c in seller_df_uncleaned.columns])


### Replacing column on product dataset with content from product category translation dataset

In [6]:
# left join between the 'product_df_uncleaned' DataFrame and 'product_category_translation_df'
product_joined_df= product_df_uncleaned.join(product_category_translation_df, "Product_category_name", "left")

# Drop "product_category_name" will be removed from the DataFrame.
product_df = product_joined_df.drop("product_category_name")

# Rename the "product_category_name_english" column to "product_category_name"
product_df = product_df.withColumnRenamed("product_category_name_english", "product_category_name")

# Replace underscores with spaces in the "product_category_name" column
product_df = product_df.withColumn("product_category_name", f.regexp_replace(f.col("product_category_name"), "_", " "))

# Show the 'product_df' DataFrame with the dropped and renamed columns.
product_df.show()

+--------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+---------------------+
|          product_id|product_name_lenght|product_description_lenght|product_photos_qty|product_weight_g|product_length_cm|product_height_cm|product_width_cm|product_category_name|
+--------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+---------------------+
|1e9e8ef04dbcff454...|                 40|                       287|                 1|             225|               16|               10|              14|            perfumery|
|3aa071139cb16b67c...|                 44|                       276|                 1|            1000|               30|               18|              20|                  art|
|96bd76ec8810374ed...|                 46|                       250|                 1|       

### Defining 0 for not_defined payment

In [7]:
# Set payment_installment to 0 where payment_type is "not_defined"
order_payment_df = order_payment_df.withColumn("Payment_installments",
                                   f.when(f.col("Payment_type") == "not_defined", 0)
                                   .otherwise(f.col("Payment_installments")))


### Additional cleaning

In [8]:
reviews_df=reviews_df.withColumn("review_score", reviews_df["review_score"].cast("int"))

In [9]:
# Replace 'reviews_df' with your actual DataFrame name
reviews_df = reviews_df.withColumn("review_comment_title", f.coalesce(f.col("review_comment_title"), f.lit("no comment")))
reviews_df = reviews_df.withColumn("review_comment_message", f.coalesce(f.col("review_comment_message"), f.lit("no comment")))

# Show the updated DataFrame
reviews_df.show()

+----+--------------------+--------------------+------------+--------------------+----------------------+--------------------+-----------------------+
|s_no|           review_id|            order_id|review_score|review_comment_title|review_comment_message|review_creation_date|review_answer_timestamp|
+----+--------------------+--------------------+------------+--------------------+----------------------+--------------------+-----------------------+
|   0|7bc2406110b926393...|73fc7af87114b3971...|           4|          no comment|            no comment| 2018-01-18 00:00:00|    2018-01-18 21:46:59|
|   1|80e641a11e56f04c1...|a548910a1c6147796...|           5|          no comment|            no comment| 2018-03-10 00:00:00|    2018-03-11 03:05:13|
|   2|228ce5500dc1d8e02...|f9e4b658b201a9f2e...|           5|          no comment|            no comment| 2018-02-17 00:00:00|    2018-02-18 14:36:24|
|   3|e64fb393e7b32834b...|658677c97b385a9be...|           5|          no comment|  I received

In [10]:
reviews_df =reviews_df.na.drop()
reviews_df.count()

96515

In [19]:
reviews_df.select("review_score").distinct().show()

+------------+
|review_score|
+------------+
|           1|
|           3|
|           5|
|           4|
|           2|
+------------+



In [12]:
reviews_df.show()

+----+--------------------+--------------------+------------+--------------------+----------------------+--------------------+-----------------------+
|s_no|           review_id|            order_id|review_score|review_comment_title|review_comment_message|review_creation_date|review_answer_timestamp|
+----+--------------------+--------------------+------------+--------------------+----------------------+--------------------+-----------------------+
|   0|7bc2406110b926393...|73fc7af87114b3971...|           4|          no comment|            no comment| 2018-01-18 00:00:00|    2018-01-18 21:46:59|
|   1|80e641a11e56f04c1...|a548910a1c6147796...|           5|          no comment|            no comment| 2018-03-10 00:00:00|    2018-03-11 03:05:13|
|   2|228ce5500dc1d8e02...|f9e4b658b201a9f2e...|           5|          no comment|            no comment| 2018-02-17 00:00:00|    2018-02-18 14:36:24|
|   3|e64fb393e7b32834b...|658677c97b385a9be...|           5|          no comment|  I received

In [13]:
reviews_df=reviews_df.drop("s_no")
reviews_df.show()


+--------------------+--------------------+------------+--------------------+----------------------+--------------------+-----------------------+
|           review_id|            order_id|review_score|review_comment_title|review_comment_message|review_creation_date|review_answer_timestamp|
+--------------------+--------------------+------------+--------------------+----------------------+--------------------+-----------------------+
|7bc2406110b926393...|73fc7af87114b3971...|           4|          no comment|            no comment| 2018-01-18 00:00:00|    2018-01-18 21:46:59|
|80e641a11e56f04c1...|a548910a1c6147796...|           5|          no comment|            no comment| 2018-03-10 00:00:00|    2018-03-11 03:05:13|
|228ce5500dc1d8e02...|f9e4b658b201a9f2e...|           5|          no comment|            no comment| 2018-02-17 00:00:00|    2018-02-18 14:36:24|
|e64fb393e7b32834b...|658677c97b385a9be...|           5|          no comment|  I received it wel...| 2017-04-21 00:00:00|   

In [14]:
reviews_df.show()

+--------------------+--------------------+------------+--------------------+----------------------+--------------------+-----------------------+
|           review_id|            order_id|review_score|review_comment_title|review_comment_message|review_creation_date|review_answer_timestamp|
+--------------------+--------------------+------------+--------------------+----------------------+--------------------+-----------------------+
|7bc2406110b926393...|73fc7af87114b3971...|           4|          no comment|            no comment| 2018-01-18 00:00:00|    2018-01-18 21:46:59|
|80e641a11e56f04c1...|a548910a1c6147796...|           5|          no comment|            no comment| 2018-03-10 00:00:00|    2018-03-11 03:05:13|
|228ce5500dc1d8e02...|f9e4b658b201a9f2e...|           5|          no comment|            no comment| 2018-02-17 00:00:00|    2018-02-18 14:36:24|
|e64fb393e7b32834b...|658677c97b385a9be...|           5|          no comment|  I received it wel...| 2017-04-21 00:00:00|   

In [18]:
reviews_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- review_score: integer (nullable = true)
 |-- review_comment_title: string (nullable = false)
 |-- review_comment_message: string (nullable = false)
 |-- review_creation_date: string (nullable = true)
 |-- review_answer_timestamp: string (nullable = true)



### Storing in parquet file

In [16]:
customer_df.coalesce(1).write.parquet("data_cleaned/customer.parquet",compression ="snappy", mode="overwrite") 
order_item_df.coalesce(1).write.parquet("./data_cleaned/order_item.parquet",compression ="snappy", mode="overwrite")
order_payment_df.coalesce(1).write.parquet("./data_cleaned/order_payment.parquet",compression ="snappy", mode="overwrite")
seller_df.coalesce(1).write.parquet("./data_cleaned/seller.parquet",compression ="snappy", mode="overwrite")
product_df.coalesce(1).write.parquet("./data_cleaned/product.parquet",compression ="snappy", mode="overwrite")
orders_df.coalesce(1).write.parquet("./data_cleaned/orders.parquet",compression ="snappy", mode="overwrite")
reviews_df.coalesce(1).write.parquet("./data_cleaned/reviews.parquet",compression ="snappy", mode="overwrite")

### Writing the clean Dataframes to Postgres

In [17]:
customer_df.write.format('jdbc').options(url='jdbc:postgresql://localhost:5432/Cleaned_Data',
                                driver = 'org.postgresql.Driver',
                                dbtable = 'customer', 
                                user='postgres', 
                                password='postgres').mode('overwrite').save()

order_item_df.write.format('jdbc').options(url='jdbc:postgresql://localhost:5432/Cleaned_Data',
                                driver = 'org.postgresql.Driver',
                                dbtable = 'order_item', 
                                user='postgres', 
                                password='postgres').mode('overwrite').save()

order_payment_df.write.format('jdbc').options(url='jdbc:postgresql://localhost:5432/Cleaned_Data',
                                driver = 'org.postgresql.Driver',
                                dbtable = 'order_payment', 
                                user='postgres', 
                                password='postgres').mode('overwrite').save()

seller_df.write.format('jdbc').options(url='jdbc:postgresql://localhost:5432/Cleaned_Data',
                                driver = 'org.postgresql.Driver',
                                dbtable = 'seller', 
                                user='postgres', 
                                password='postgres').mode('overwrite').save()

product_df.write.format('jdbc').options(url='jdbc:postgresql://localhost:5432/Cleaned_Data',
                                driver = 'org.postgresql.Driver',
                                dbtable = 'product', 
                                user='postgres', 
                                password='postgres').mode('overwrite').save()

orders_df.write.format('jdbc').options(url='jdbc:postgresql://localhost:5432/Cleaned_Data',
                                driver = 'org.postgresql.Driver',
                                dbtable = 'orders', 
                                user='postgres', 
                                password='postgres').mode('overwrite').save()

reviews_df.write.format('jdbc').options(url='jdbc:postgresql://localhost:5432/Cleaned_Data',
                                driver = 'org.postgresql.Driver',
                                dbtable = 'reviews', 
                                user='postgres', 
                                password='postgres').mode('overwrite').save()